### Load the data

In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import datetime
import matplotlib.pyplot as plt

from helpers import *
from implementations import *
from Data_Cleaning import *

%load_ext autoreload
%autoreload 2

In [ ]:
y, tx, ids = load_csv_data("train.csv", sub_sample=False)
for i in range(tx.shape[1]):
    print(tx[:, i][tx[:, 22] == 0][(tx[tx[:, 22] == 0][:, i] == -999)].shape)
    

In [ ]:
def correctOutliers(xi, lower, upper, newVals):
    a = xi > upper
    b = xi < lower
    aorb = np.logical_or(a,b)
    xxi = np.where(aorb, newVals, xi)
    return xxi

def handleOutliers(tx):
    q3, q2, q1 = np.quantile(tx, [0.75, 0.5, 0.25], axis=0)
    iqr = q3 - q1
    upper = q3 + 1.5*iqr
    lower = q1 - 1.5*iqr
    tx2 = np.apply_along_axis(correctOutliers, axis=1, arr=tx, lower=lower, upper=upper, newVals=q2)
    return tx2

In [ ]:
y, tx, ids = load_csv_data("train.csv", sub_sample=False)
for i in range(tx.shape[1]):
    tx[:, i][tx[:, i] == -999] = np.mean(tx[:, i][tx[:, i] != -999])
data_model = split_according_num_split_jet(y,tx,ids)

In [ ]:
tx = data_model["0"]["data"]
ind = data_model["0"]["ind"]
y = data_model["0"]["label"]

tx_mean = np.mean(tx, axis=0)
tx_std = np.std(tx, axis=0)
tx = (tx - tx_mean) / tx_std

In [ ]:
np.where(0==tx_mean)

In [ ]:
y, tx, ids = load_csv_data("train.csv", sub_sample=False)

# change outlier values to the mean without them
tx_mean_meaningful = np.zeros([tx.shape[1],])
for i in range(tx.shape[1]):
    tx[:, i][tx[:, i] == -999] = np.mean(tx[:, i][tx[:, i] != -999])
    mean.append(np.mean(tx[:, i][tx[:, i] != -999]))
#print(tx[:,0].mean(),min(tx[:,0]))
# transform the categorical features into one-hot features
#tx_ = np.delete(tx, 22, axis = 1)
#tx = np.hstack((tx,tx_**2,tx_**3,np.sin(tx_),np.cos(tx_),np.sin(2*tx_),np.cos(2*tx_),np.sin(tx_)**2,np.cos(tx_)**2,np.sin(0.5*tx_),np.cos(0.5*tx_),
#    (tx[:, 22] == 0).astype(np.float64)[:, np.newaxis],
#    (tx[:, 22] == 1).astype(np.float64)[:, np.newaxis],
#    (tx[:, 22] == 2).astype(np.float64)[:, np.newaxis],
#    (tx[:, 22] == 3).astype(np.float64)[:, np.newaxis]))
i = -1
ind = np.where(tx[:, 22] > i)
tx_ = np.delete(tx[tx[:, 22] > i], 22, axis = 1)
#tx = np.hstack([tx[tx[:, 22] > 1],np.sin(tx_),np.cos(tx_)])#,np.sin(2*tx_),np.cos(2*tx_)])
tx = np.delete(tx[tx[:, 22] > i], 22, axis = 1)
#tx = build_derived_quantities(tx)
tx = build_poly_cross_terms(tx, 3)
# standardization
tx_mean = np.mean(tx, axis=0)
tx_std = np.std(tx, axis=0) 
tx[:, :-4] = (tx[:, :-4] - tx_mean[:-4]) / tx_std[:-4]

# sin, cos term # improvement ~ 1%
tx = np.hstack([tx, np.cos(tx),np.sin(2*tx),np.cos(2*tx)])

# drop all-zero columns, which may cause tx.T @ tx a singular matrix
zero_idx = np.argwhere(np.all(tx == 0, axis=0))
tx = np.delete(tx, zero_idx, axis=1)

# add the bias term to features
bias_term = np.ones([tx.shape[0], 1])
tx = np.hstack([tx, bias_term])

# drop duplicate columns
tx, unique_idx = np.unique(tx, axis=1, return_index=True)

In [ ]:
tx.shape, y.shape

In [ ]:
# y, tx, ids = load_csv_data("train.csv", sub_sample=False)

# # change outlier values to the mean without them
# mean = []
# for i in range(tx.shape[1]):
#     tx[:, i][tx[:, i] == -999] = np.mean(tx[:, i][tx[:, i] != -999])
#     mean.append(np.mean(tx[:, i][tx[:, i] != -999]))
# # transform the categorical features into one-hot features
# #tx_ = np.delete(tx, 22, axis = 1)
# #tx = np.hstack((tx,tx_**2,tx_**3,np.sin(tx_),np.cos(tx_),np.sin(2*tx_),np.cos(2*tx_),np.sin(tx_)**2,np.cos(tx_)**2,np.sin(0.5*tx_),np.cos(0.5*tx_),
# #    (tx[:, 22] == 0).astype(np.float64)[:, np.newaxis],
# #    (tx[:, 22] == 1).astype(np.float64)[:, np.newaxis],
# #    (tx[:, 22] == 2).astype(np.float64)[:, np.newaxis],
# #    (tx[:, 22] == 3).astype(np.float64)[:, np.newaxis]))
# ind = np.where(tx[:, 22] > 1)
# tx_ = np.delete(tx[tx[:, 22] > 1], 22, axis = 1)
# tx__ = np.hstack([tx[tx[:, 22] > 1],tx_[:,0].reshape((tx_.shape[0],1))*tx_,tx_[:,1].reshape((tx_.shape[0],1))*tx_,tx_[:,2].reshape((tx_.shape[0],1))*tx_,tx_[:,3].reshape((tx_.shape[0],1))*tx_,tx_[:,4].reshape((tx_.shape[0],1))*tx_,tx_[:,5].reshape((tx_.shape[0],1))*tx_,tx_[:,6].reshape((tx_.shape[0],1))*tx_,
#                 tx_[:,7].reshape((tx_.shape[0],1))*tx_,np.sin(tx_),tx_[:,8].reshape((tx_.shape[0],1))*tx_,
#                 tx_[:,9].reshape((tx_.shape[0],1))*tx_,tx_[:,10].reshape((tx_.shape[0],1))*tx_,
#                 tx_[:,11].reshape((tx_.shape[0],1))*tx_,tx_[:,12].reshape((tx_.shape[0],1))*tx_,tx_[:,13].reshape((tx_.shape[0],1))*tx_,
#                 tx_[:,14].reshape((tx_.shape[0],1))*tx_,tx_[:,15].reshape((tx_.shape[0],1))*tx_,tx_[:,16].reshape((tx_.shape[0],1))*tx_,
#                 tx_[:,17].reshape((tx_.shape[0],1))*tx_,tx_[:,18].reshape((tx_.shape[0],1))*tx_,tx_[:,19].reshape((tx_.shape[0],1))*tx_,
#                 np.cos(tx_),np.sin(2*tx_),np.cos(2*tx_)])
# tx = np.hstack([tx_])
# tx = np.delete(tx, 22, axis = 1)

# # standardization
# tx_mean = np.mean(tx, axis=0)
# tx_std = np.std(tx, axis=0)
# tx = (tx - tx_mean) / tx_std

In [ ]:
# #y.shape, tx.shape
# np.mean(tx, axis = 0).shape


### Gradient Descent

In [ ]:
# Define the parameters of the algorithm.
max_iters = 100
gamma = 0.005

# Initialization
initial_w = gd_ws#np.zeros(tx.shape[1])

# Start gradient descent.
start_time = datetime.datetime.now()
gd_ws, gd_losses= least_squares_GD(y, tx, initial_w, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Gradient Descent: execution time = {t:.3f} seconds".format(t=exection_time))
print("Gradient Descent: loss = {t}".format(t=gd_losses))

y_pred = tx @ gd_ws
y_pred[y_pred > 0] = 1
y_pred[y_pred < 0] = -1
print('Gradient descent accuracy:', np.mean(y_pred == y))

In [ ]:
# cross validation for GD. Maybe No Need...

from helpers import *

k_fold = 5
seed = None
initial_w = np.zeros(tx.shape[1])
max_iters = 10 
gammas = np.logspace(-4, -2, 20)

# select gamma
gamma_ = hyperparameter_tuning_GD(tx, y, k_fold, seed, initial_w, max_iters, gammas)

# Start gd
gd_ws, gd_losses = least_squares_GD(y, tx, initial_w, max_iters, gamma_)

# Print result
print("Hyperparameter Selection: Gamma = {t}".format(t=gamma_))
print("Gradient Descent: loss = {t}".format(t=gd_losses))

y_pred = tx @ gd_ws
y_pred[y_pred > 0] = 1
y_pred[y_pred < 0] = -1
print('Gradient descent accuracy:', np.mean(y_pred == y))

### Stochastic Gradient Descent

In [ ]:
# Define the parameters of the algorithm.
max_iters = 10
gamma = 0.000000015
sgd_ws = np.zeros(tx.shape[1])

# Initialization
initial_w = sgd_ws

# Start gradient descent.
start_time = datetime.datetime.now()
sgd_ws, sgd_losses= least_squares_SGD(y, tx, initial_w, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Stochastic Gradient Descent: execution time = {t:.3f} seconds".format(t=exection_time))
print("Stochastic Gradient Descent: loss = {t}".format(t=sgd_losses))

y_pred = tx @ sgd_ws
y_pred[y_pred > 0] = 1
y_pred[y_pred < 0] = -1
print('Stochastic gradient descent accuracy:', np.mean(y_pred == y))

### Least Squares

In [ ]:
# Start least squares.
start_time = datetime.datetime.now()
ls_ws, ls_losses= least_squares(y, tx)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Least Squares: execution time = {t:.3f} seconds".format(t=exection_time))
print("Least Squares: loss = {t}".format(t=ls_losses))

y_pred = tx @ ls_ws
y_pred[y_pred > 0] = 1
y_pred[y_pred < 0] = -1
print('Least squares accuracy:', np.mean(y_pred == y[ind]))

### Ridge Regression

In [ ]:
# initialization
degree = 1
ratio = 0.8
seed = None
lambdas = np.logspace(-20, -15, 150)

# select lambda
lambda_ = hyperparameter_tuning_ridge_regression(tx, y[ind], degree, ratio, seed, lambdas)

# Start ridge regression
start_time = datetime.datetime.now()
r_ws, r_losses = ridge_regression(y[ind], tx, lambda_)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Ridge Regression: execution time = {t:.3f} seconds".format(t=exection_time))
print("Ridge Regression: loss = {t}".format(t=r_losses))

y_pred = tx @ r_ws
y_pred[y_pred > 0] = 1
y_pred[y_pred < 0] = -1
print('Ridge Regression accuracy:', np.mean(y_pred == y))


In [ ]:
# # initialization
# degree = 1
# ratio = 0.8
# seed = None
# lambdas = np.logspace(-20, -15, 150)

# # select lambda
# lambda_ = hyperparameter_tuning_ridge_regression(tx, y[ind], degree, ratio, seed, lambdas)

# # Start ridge regression
# start_time = datetime.datetime.now()
# r_ws, r_losses = ridge_regression(y[ind], tx, lambda_)
# end_time = datetime.datetime.now()

# # Print result
# exection_time = (end_time - start_time).total_seconds()
# print("Ridge Regression: execution time = {t:.3f} seconds".format(t=exection_time))
# print("Hyperparameter Selection: Lambda = {t}".format(t=lambda_))
# print("Ridge Regression: loss = {t}".format(t=r_losses))

In [ ]:
# def build_k_indices(y, k_fold, seed):
#     """build k indices for k-fold.
#     return dimension: k-fold * interval
#     """
#     num_row = y.shape[0]
#     interval = int(num_row / k_fold)
#     np.random.seed(seed)
#     indices = np.random.permutation(num_row)
#     k_indices = [indices[k * interval: (k + 1) * interval] for k in range(k_fold)] 
#     return np.array(k_indices)

In [ ]:
# # Cross Validation

# Initialization
seed = None
k_fold = 5
k_indices = build_k_indices(y[ind], k_fold, seed)

train_acc = []
val_acc = []
for k in range(k_fold):
    val_y = y[ind][k_indices[k, :]]
    val_x = tx[k_indices[k, :]]
    train_idx = np.vstack([k_indices[:k, :], k_indices[k+1:, :]]).flatten()
    train_y = y[train_idx]
    train_x = tx[train_idx]

    print('------', k, 'fold ------')
    w, loss = ridge_regression(train_y, train_x, lambda_)
    pred = train_x @ w
    pred[pred > 0] = 1
    pred[pred < 0] = -1
    train_acc.append(np.mean(pred == train_y))
    print('Train acc:', np.mean(pred == train_y))
    pred = val_x @ w
    pred[pred > 0] = 1
    pred[pred < 0] = -1
    val_acc.append(np.mean(pred == val_y))
    print('Validation acc:', np.mean(pred == val_y))

print('------ summary ------')
print('Average Train acc:', sum(train_acc)/len(train_acc))
print('Average Validation acc:', sum(val_acc)/len(val_acc))

## Logistic Regression

In [ ]:
# Define the parameters of the algorithm.
max_iters = 2
gamma = 0.00005


# Initialization
initial_w = gd_ws#np.zeros((tx.shape[1]))

# Start gradient descent.
start_time = datetime.datetime.now()
gd_ws, gd_losses= logistic_regression(y[ind], tx, initial_w, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Gradient Descent: execution time = {t:.3f} seconds".format(t=exection_time))
print("Gradient Descent: loss = {t}".format(t=gd_losses))

In [ ]:
def predict_labels_logit(w, tx):
    y_pred = sigmoid(np.dot(tx, w))
    y_pred[np.where(y_pred <= 0.5)] = -1
    y_pred[np.where(y_pred > 0.5)] = 1
    return y_pred
# predictions
y_pred = predict_labels_logit(gd_ws, tx)

# accuracy
(y_pred == y[ind]).mean()

## Regularized logistic regression

In [ ]:
# Define the parameters of the algorithm.
max_iters = 4
gamma = 0.0001
lambda_ = 0.000001

# Initialization
initial_w = np.random.normal(scale = 1/np.sqrt(tx.shape[0]), size=(tx.shape[1]))

# Start gradient descent.
start_time = datetime.datetime.now()
gd_ws, gd_losses= reg_logistic_regression(y[ind], tx,lambda_, initial_w, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Gradient Descent: execution time = {t:.3f} seconds".format(t=exection_time))
print("Gradient Descent: loss = {t}".format(t=gd_losses))

In [ ]:
# predictions
y_pred = predict_labels_logit(gd_ws, tx)

# accuracy
(y_pred == y[ind]).mean()

In [ ]:
#reg_logit_loss(y,tx,lambda_,gd_ws)
#(sigmoid(tx@gd_ws)<0).sum()
#tx@gd_ws

In [ ]:

xx = build_poly(tx[:, 1:], 2)[:, 1:]
print(xx)
w_init = np.zeros(xx.shape[1])
train_first_feature(tx[:, 0], xx, w_init, 0.0001, 20, 0)

### Prediction

#### To start, you shall upload text.csv to the branch

In [ ]:
_, tx_test, ids_test = load_csv_data("test.csv", sub_sample=False)

# force meaningless data into the means
for i in range(tx_test.shape[1]):
    tx_test[:, i][tx_test[:, i] == -999] = tx_mean_meaningful[i]

# transform the categorical feature (PRI_jet_num) into onthot features
tx_test = np.hstack((tx_test, 
    (tx_test[:, 22] == 0).astype(np.float64)[:, np.newaxis],
    (tx_test[:, 22] == 1).astype(np.float64)[:, np.newaxis],
    (tx_test[:, 22] == 2).astype(np.float64)[:, np.newaxis],
    (tx_test[:, 22] == 3).astype(np.float64)[:, np.newaxis]))
tx_test = np.delete(tx_test, 22, axis = 1)

# standardization
tx_test_mean = np.mean(tx_test, axis=0)
tx_test_std = np.std(tx_test, axis=0)
tx_test[:, :-4] = (tx_test[:, :-4] - tx_test_mean[:-4]) / tx_test_std[:-4]

# sin, cos term # improvement ~
tx_test = np.hstack([tx_test, np.cos(tx_test),np.sin(2*tx_test),np.cos(2*tx_test)])

# drop all-zero columns, which may cause tX.T @ tX a singular matrix
zero_idx = np.argwhere(np.all(tx_test == 0, axis=0))
tx_test = np.delete(tx_test, zero_idx, axis=1)

# add the bias term to features
bias_term = np.ones([tx_test.shape[0], 1])
tx_test = np.hstack([tx_test, bias_term])

# drop duplicate columns
tx_test = tx_test[:, unique_idx]

In [ ]:
import itertools

for ind in itertools.combinations_with_replacement(np.nditer(tx,op_axes = 0),2):
    print(ind)